In [1]:
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dill
from typing import List
import scipy.stats as sts
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

import util
from pathlib import Path
from class_manipulates_path import ManipulatePath
from class_preprocessing_refactor import Preprocessing
from class_format_data import FormatData
util.init()

current_path = sys.path[0]

In [2]:
def plot_variable(df, column_name, intervalos=None):
    """
    Plota um gráfico de dispersão para uma variável de um DataFrame.

    Parâmetros:
        - df: DataFrame pandas contendo os dados.
        - column_name: Nome da coluna que você deseja plotar.
        - intervalos: Lista de intervalos para plotar linhas verticais.
    """
    plt.figure(figsize=(20, 10))
    plt.scatter(df.index.values, df[column_name].values, color='darkcyan', alpha=0.5)
    plt.xlabel('Índice')
    plt.ylabel(column_name)
    plt.title(f'Gráfico de dispersão para a variável "{column_name}"')


    # Plotar linhas verticais para cada intervalo
    if intervalos:
        for intervalo in intervalos:
            plt.axvline(x=intervalo, linestyle='--', color='red')

    plt.show()

In [3]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [4]:
manipulate_path = ManipulatePath()

preprocessing = Preprocessing()

format_data = FormatData()

path_raw_data = manipulate_path.get_path_raw_data()

abnormal_classes_codes = [1, 2, 5, 6, 7, 8]

In [5]:
# Gets all real instances but maintains only those with any type of undesirable event
real_instances = pd.DataFrame(class_and_file_generator(path_raw_data, 
                                                       real=True,
                                                       simulated=True, 
                                                       drawn=True),
                              columns=['class_code', 'instance_path'])
real_instances = real_instances.loc[real_instances.iloc[:,0].isin(abnormal_classes_codes)].reset_index(drop=True)

In [6]:
falha = 6
falha_2 = 106

In [7]:
real_instances[real_instances["class_code"] == falha]

,class_code,instance_path
617,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
618,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
619,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
620,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
621,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
...,...,...
833,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
834,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
835,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
836,6,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...


In [8]:
real_instances[real_instances["class_code"] == falha]["instance_path"].values

array([WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/6/SIMULATED_00001.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/6/SIMULATED_00002.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/6/SIMULATED_00003.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/6/SIMULATED_00004.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/6/SIMULATED_00005.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/6/SIMULATED_00006.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Fina

In [9]:
file_paths = real_instances[real_instances["class_code"] == falha]["instance_path"].values
df_output = pd.DataFrame()

for file_path in file_paths:
    df_data = pd.read_csv(file_path)

    df_data = FormatData.set_index_dataframe(df_data=df_data,
                                         column_name="timestamp")
    quantidade = ((df_data['class'] == falha) | (df_data['class'] == falha_2)).sum()
    if quantidade != 0:
        print(f"1 se repete {quantidade}")
        print(f"Index min {df_data.index.min()} index max {df_data.index.max()}")

    df_output = pd.concat([df_output, df_data], axis=0)

1 se repete 25199
Index min 2018-04-01 00:45:25 index max 2018-04-01 08:15:23
1 se repete 25199
Index min 2018-04-01 05:56:40 index max 2018-04-01 13:26:38
1 se repete 25199
Index min 2018-04-01 07:22:46 index max 2018-04-01 14:52:44
1 se repete 25199
Index min 2018-04-01 09:15:05 index max 2018-04-01 16:45:03
1 se repete 25200
Index min 2018-04-01 12:16:31 index max 2018-04-01 19:46:29
1 se repete 25199
Index min 2018-04-01 13:20:14 index max 2018-04-01 20:50:12
1 se repete 25199
Index min 2018-04-01 14:22:31 index max 2018-04-01 21:52:29
1 se repete 25199
Index min 2018-04-01 15:20:34 index max 2018-04-01 22:50:32
1 se repete 25199
Index min 2018-04-01 18:25:43 index max 2018-04-02 01:55:41
1 se repete 25199
Index min 2018-04-01 17:47:35 index max 2018-04-02 01:17:33
1 se repete 25199
Index min 2018-04-01 18:11:51 index max 2018-04-02 01:41:49
1 se repete 25199
Index min 2018-04-01 18:45:12 index max 2018-04-02 02:15:10
1 se repete 25199
Index min 2018-04-01 19:02:13 index max 2018-0

In [10]:
df_output

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
timestamp,,,,,,,,,
2018-04-01 00:45:25,21077800.0,14301040.0,NaN,2609378.0,84.11474,NaN,NaN,NaN,0.0
2018-04-01 00:45:26,21077770.0,14301020.0,NaN,2609378.0,84.11474,NaN,NaN,NaN,0.0
2018-04-01 00:45:27,21077750.0,14301110.0,NaN,2609378.0,84.11474,NaN,NaN,NaN,0.0
2018-04-01 00:45:28,21077730.0,14301030.0,NaN,2609378.0,84.11473,NaN,NaN,NaN,0.0
2018-04-01 00:45:29,21077720.0,14301040.0,NaN,2609378.0,84.11473,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...
2017-10-31 20:28:52,0.0,13364320.0,106.8570,5758852.0,72.82104,NaN,NaN,NaN,6.0
2017-10-31 20:28:53,0.0,13365090.0,106.8581,5759003.0,72.82066,NaN,NaN,NaN,6.0
2017-10-31 20:28:54,0.0,13365850.0,106.8592,5759155.0,72.82028,NaN,NaN,NaN,6.0


In [11]:
df_output = df_output[~df_output.index.duplicated()]

In [12]:
df_output.shape

(369081, 9)

In [13]:
df_aux = df_output.drop(columns=["QGL", "T-JUS-CKGL"])
df_aux

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,class
timestamp,,,,,,,
2018-04-01 00:45:25,21077800.0,14301040.0,NaN,2609378.0,84.11474,NaN,0.0
2018-04-01 00:45:26,21077770.0,14301020.0,NaN,2609378.0,84.11474,NaN,0.0
2018-04-01 00:45:27,21077750.0,14301110.0,NaN,2609378.0,84.11474,NaN,0.0
2018-04-01 00:45:28,21077730.0,14301030.0,NaN,2609378.0,84.11473,NaN,0.0
2018-04-01 00:45:29,21077720.0,14301040.0,NaN,2609378.0,84.11473,NaN,0.0
...,...,...,...,...,...,...,...
2017-10-31 20:28:52,0.0,13364320.0,106.8570,5758852.0,72.82104,NaN,6.0
2017-10-31 20:28:53,0.0,13365090.0,106.8581,5759003.0,72.82066,NaN,6.0
2017-10-31 20:28:54,0.0,13365850.0,106.8592,5759155.0,72.82028,NaN,6.0


In [14]:
for column_name in df_aux.columns:
    percent = Preprocessing.check_null_dataframe(df_data=df_output, column_name=column_name)
    print(f"{column_name} null data is {percent}")

P-PDG null data is 0.0
P-TPT null data is 0.0
T-TPT null data is 84.85508601092985
P-MON-CKP null data is 0.0
T-JUS-CKP null data is 0.0
P-JUS-CKGL null data is 87.03021829896417
class null data is 0.17882253489071503


In [15]:
df_aux = df_aux.dropna()
df_aux.shape

(47386, 7)

In [16]:
(df_aux['class'] == 0).sum() / df_aux.shape[0]

0.6436078166547081

# KNN

In [17]:
contagem_classes = df_aux['class'].value_counts()

# Calcular a porcentagem de cada classe
porcentagem_classes = contagem_classes / len(df_aux) * 100
porcentagem_classes

class
0.0      64.360782
6.0      26.225045
106.0     9.414173
Name: count, dtype: float64

In [18]:
# Separar as features (X) e os rótulos (y)
X = df_aux.drop('class', axis=1)
y = df_aux['class']

In [19]:
X

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL
timestamp,,,,,,
2014-02-12 17:03:33,0.0,12804630.0,118.7102,2514857.0,83.49664,4802594.0
2014-02-12 17:03:34,0.0,12804800.0,118.7104,2514857.0,83.49661,4802581.0
2014-02-12 17:03:35,0.0,12804970.0,118.7106,2514857.0,83.49659,4802568.0
2014-02-12 17:03:36,0.0,12805130.0,118.7108,2514856.0,83.49656,4802555.0
2014-02-12 17:03:37,0.0,12805300.0,118.7110,2514856.0,83.49654,4802542.0
...,...,...,...,...,...,...
2014-03-25 20:23:20,0.0,14055750.0,118.5514,4591160.0,80.85170,4800835.0
2014-03-25 20:23:21,0.0,14055830.0,118.5513,4591190.0,80.85181,4800767.0
2014-03-25 20:23:22,0.0,14055910.0,118.5511,4591219.0,80.85191,4800699.0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [22]:
modelo = KNeighborsClassifier(n_neighbors=5)

In [23]:
modelo.fit(X_train, y_train)

KNeighborsClassifier()

In [24]:
y_pred = modelo.predict(X_test)

In [25]:
accuracy_score(y_test, y_pred)

1.0

In [26]:
precision = precision_score(y_test,  y_pred, average=None)
recall = recall_score(y_test,  y_pred, average=None)
f1 = f1_score(y_test,  y_pred, average=None)

for i in range(len(precision)):
    print(f'Classe {porcentagem_classes.index.values[i]}:')
    print(f'Precision: {precision[i]:.3f}')
    print(f'Recall: {recall[i]:.3f}')
    print(f'F1-score: {f1[i]:.3f}\n')

Classe 0.0:
Precision: 1.000
Recall: 1.000
F1-score: 1.000

Classe 6.0:
Precision: 1.000
Recall: 1.000
F1-score: 1.000

Classe 106.0:
Precision: 1.000
Recall: 1.000
F1-score: 1.000



In [27]:
confusion_matrix(y_test.values, y_pred)

array([[9150,    0,    0],
       [   0, 3728,    0],
       [   0,    0, 1338]], dtype=int64)

# MLP

In [27]:
modelo = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [28]:
modelo.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [29]:
y_pred = modelo.predict(X_test)

In [30]:
accuracy_score(y_test, y_pred)

1.0